Install the necessary libraries - Tested with Python 3.10

In [1]:
!pip install numpy==1.24.2 torch==2.0.0 anytree transformers==4.27.4 safetensors sentencepiece

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 2.0 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 3.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 MB 2.7 MB/s eta 0:00:0000:0100:02
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 2.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.9/44.9 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 415.8/415.8 kB 2.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 402.6/402.6 kB 2.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.4/189.4 kB 3.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.7/281.7 kB 2.9 MB/s 

Import the necessary libraries

In [1]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
import torch
from lrml import *

/Users/stefanfuchs/Repos/lrml-editor/.conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Load the model from Huggingface

In [2]:
model_path = 'sffc348/t5-base-lrml-autocomplete'
model = T5ForConditionalGeneration.from_pretrained('sffc348/t5-base-lrml-autocomplete', use_safetensors=True)
model.eval()

/Users/stefanfuchs/Repos/lrml-editor/.conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dro

In [3]:
tokenizer_name = 't5-base'

def load_tokenizer():
    tokenizer = T5Tokenizer.from_pretrained(tokenizer_name)
    tokenizer.add_tokens(['<sep>'], special_tokens=True)
    tokenizer.sep_token = '<sep>'
    tokenizer.sep_token_id = tokenizer.convert_tokens_to_ids(
        tokenizer.sep_token)
    return tokenizer

tokenizer = load_tokenizer()

/Users/stefanfuchs/Repos/lrml-editor/.conda/lib/python3.10/site-packages/transformers/models/t5/tokenization_t5.py:163: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [4]:
# Preprocessing of text
def normalise_text(text):
    text = text.strip()
    if text and text[-1] != '.':
        text += '.'
    return text

# Generate a prediction based on the input text and the LRML
def predict(text, lrml):
    num_beams = 5
    num_return_sequences = 5
    no_repeat_ngram_size = 8
    max_length = 256
    early_stopping = True
    print(text, lrml)
    if lrml.strip() != '':
        lrml = '<sep>' + lrml
    else:
        lrml = ''
    tokens = tokenizer('translate English to LegalRuleML: ' +
                            normalise_text(text) + lrml, return_tensors='pt')
    with torch.no_grad():
        generation = model.generate(inputs=tokens.input_ids, max_length=max_length, num_beams=num_beams,
                                            num_return_sequences=num_return_sequences, early_stopping=early_stopping,
                                            no_repeat_ngram_size=no_repeat_ngram_size)

    return [post_process(i) for i in tokenizer.batch_decode(generation, skip_special_tokens=True)]

# Functions for postprocessing
def post_process(lrml):
    lrml = lrml.strip()
    # lrml = lrml[lrml.find('if('):]
    lrml = lrml.replace('[', '(').replace(']', ')').replace(
        '{', '(').replace('}', ')')
    lrml = lrml.replace(').', ')')
    lrml = fix_then(lrml, ' ')
    lrml = revert_tree_based_spacing(lrml)
    lrml = add_space_after_comma(lrml)

    return lrml

def clean_pred(lrml):
    prefix = ''
    lrml = lrml.replace(', ', ',')

    lrml = reverse_loop(lrml, prefix=prefix)
    lrml = reverse_resolve_expressions(lrml, fix_errors=True, prefix=prefix)
    lrml = reverse_combine_rel_and_var(lrml, prefix=prefix)
    lrml = reverse_move_and_or_to_data_node(lrml)
    lrml = reverse_units(lrml, prefix=prefix)
    return lrml

Do the actual prediction, the LRML input decides the scope of the prediction

In [5]:
text = 'The floor waste shall have a minimum diameter of 40 mm.'
# lrml = ''
lrml = 'if('
# lrml = 'if(exist(floorWaste)), then(obligation('
predictions = predict(text, lrml)
predictions

The floor waste shall have a minimum diameter of 40 mm. if(


['exist(floorWaste)',
 'and(has(floorWaste, diameter), greaterThanEqual(diameter, 40 mm))',
 'and(greaterThanEqual(floorWaste.diameter, 40 mm))',
 'and(has(floorWaste, diameter), greaterThanEqual(diameter, 40 mm))',
 'and(has(floorWaste, diameter), greaterThanEqual(diameter, 40 mm))']

Post-process the output

In [6]:
clean_pred(predictions[0])

'expression(function(exist),atom(variable(floorWaste)))'